# Label BoundingBox —— Image & Video
Pre-require: [ipywidgets](https://github.com/jupyter-widgets/ipywidgets), you can install it by executing `pip install ipywidgets` in terminal.   
Usage: Replace `RECORD` & `IMGs_ROOT` with path & directory that you want to explore, run the cell, then choose images from the `Dropdown` box.

环境要求：
- 需要[ipywidgets](https://github.com/jupyter-widgets/ipywidgets)，可以使用终端命令`pip install ipywidgets`安装。

用法：
 
_在进行以下操作之前，请做好原始记录文件的备份!_
1. 先用fix_records_length.ipynb文件将不定长记录重新整理成定长记录
2. 将变量`RECORD`的值修改为定长记录文件的路径
3. 将变量`IMGs_ROOT`的值修改为记录文件中路径的相对起始位置（如果是绝对路径，则将`IMGs_ROOT`设为空字符串`''`）
4. 如果需要自动保存，将变量`AUTOSAVE`设置为`True`（这将会占用更多资源，如果数据过多，建议设为`False`）
4. 运行代码块，在`Label`文本框中输入新的label后，回车即自动加载下一张图片
5. 最后一定要**点击保存按钮**，确保结果覆盖保存到`RECORD`文件。
6. 点击`Read File`蓝色按钮重新读取文件，确认结果正确。

> **随时保存！随时保存！随时保存！**重要的事说三遍！

In [34]:
# Change RECORD as the fixed records file's path
RECORD = "./label.txt"
# Change IMGs_ROOT as the dictionary where images are
IMGs_ROOT = "./"
# Save changes after every input
AUTOSAVE = True
# Standing time for each image when AutoPlay, unit: second
LOOP_TIME = 0.5

IMAGE_FMT = [
    '.jpg', '.jpeg', '.jpe', '.png', '.bmp', '.dib', '.tif', '.tiff', '.webp',
    '.pbm', '.pgm', '.ppm', '.sr', '.ras', '.exr', '.jp2'
]
VIDEO_FMT = ['.mp4', '.avi', '.mpg', '.mpeg', '.mov']

import os
import cv2
import numpy as np
from time import sleep
from ipywidgets import Label, Text, Button, Dropdown, Image, HBox, HTML
from IPython.display import display, clear_output

idx = 0
metadata = []
info_lbl = HTML()
rec_txt = Text(description="RECORD:")
rec_txt.layout.width = "90%"
jmp_btn = Button(description="Read File", button_style='primary')
jmp_btn.layout.width = "80px"
drpbox = Dropdown(description="[0/0]:")
drpbox.layout.width = "90%"
prev_btn = Button(description="Preview")
next_btn = Button(description="Next")
loop_btn = Button(description="AutoPlay")
loop_btn.layout.width = "200px"
save_btn = Button(description="Save")
imgbox = Image(format='jpg')
imgbox.layout.max_height = "500px"
label_txt = Text(description="New label:")
label_lbl = Label()


def read_rec(_=None):
    global RECORD, idx, metadata
    idx = 0
    RECORD = rec_txt.value
    if os.path.isfile(RECORD):
        with open(RECORD) as fp:
            for rowstr in fp.readlines():
                metadata.append(pattern_row(rowstr))
    else:
        import ipdb; ipdb.set_trace()
        metadata = []
        info_lbl.value = "<font color=\"#ff000000\">{}</font>".format(
            "Error: %s is not an available path!" % RECORD)
    drpbox.options = [(value[0], i) for i, value in enumerate(metadata)]
    display_com()
    show_pic()


def pattern_row(rowstr):
    rowdata = rowstr.split()
    path = rowdata[0]
    bbox = [float(d) for d in rowdata[1:]]
    x0, y0, x1, y1 = bbox[:4]
    if len(bbox) >= 6:
        label, score =  bbox[4:6]
    else:
        label = -1
        score = -1
    return [path, int(x0), int(y0), int(x1), int(y1), str(label), score]


def show_pic():
    global idx, metadata
    prev_btn.disabled = next_btn.disabled = False
    if idx <= 0:
        idx = 0
        prev_btn.disabled = True
    if idx >= len(metadata) - 1:
        idx = max(len(metadata) - 1, 0)
        next_btn.disabled = True
    drpbox.description = "[{}/{}]:".format(idx + 1, len(metadata))
    if len(metadata) == 0:
        return None
    drpbox.value = idx
    pic_path, x0, y0, x1, y1, label, score = metadata[idx]
    crtpath = os.path.join(IMGs_ROOT, pic_path)
    _, ext = os.path.splitext(crtpath)
    if ext in IMAGE_FMT:
        img = cv2.imread(crtpath)
        img = cv2.rectangle(img, (x0, y0), (x1, y1), [0, 0, 255], 2)
        label_lbl.value = 'Preview Label: ' + str(label)
        label_txt.value = ''
        imgbox.value = cv2.imencode('.jpg', img)[1].tobytes()
    elif ext in VIDEO_FMT:
        cap = cv2.VideoCapture(crtpath)
        fps = cap.get(cv2.CAP_PROP_FPS)
        while (cap.isOpened()):
            ret, frame = cap.read()
            if not ret:
                break
            imgbox.value = cv2.imencode('.jpg', frame)[1].tobytes()
            sleep(1 / fps)
    return crtpath


def choose_pic(attval):
    global idx
    idx = attval['new']
    show_pic()


def prev_pic(sender):
    global idx
    idx -= 1
    show_pic()


def next_pic(sender):
    global idx
    idx += 1
    show_pic()


def label_enter(sender):
    global idx
    # todo: check input label
    metadata[idx][-2] = label_txt.value
    if AUTOSAVE:
        save_pic(sender)
    next_pic(sender)


def loop_pic(sender):
    if loop_btn.description == "AutoPlay":
        loop_btn.description = "Stop loop by Kernel->Interrupt"
        loop_btn.disabled = True
        global idx
        try:
            while idx <= len(metadata) - 1:
                show_pic()
                sleep(0.2)
                idx += 1
        except KeyboardInterrupt:
            pass
    loop_btn.description = "AutoPlay"
    loop_btn.disabled = False


def save_pic(sender):
    try:
        with open(RECORD, 'w+') as fo:
            for itm in metadata:
                rowstr = ' '.join([str(elm) for elm in itm])
                fo.write(rowstr + '\n')
        info_lbl.value = "<font color=\"#00ff0000\">{}</font>".format(
            "Saved into %s" % RECORD)
    except Exception as e:
        info_lbl.value = "<font color=\"#ff000000\">{}</font>".format(
            "Error: %s!" % str(e))


def display_com():
    clear_output()
    display(HBox([rec_txt, jmp_btn]), drpbox,
        HBox([prev_btn, next_btn, loop_btn, save_btn]), info_lbl, imgbox,
        HBox([label_txt, label_lbl]))
    
rec_txt.on_submit(read_rec)
jmp_btn.on_click(read_rec)
drpbox.observe(choose_pic, names='value')
prev_btn.on_click(prev_pic)
next_btn.on_click(next_pic)
label_txt.on_submit(label_enter)
loop_btn.on_click(loop_pic)
save_btn.on_click(save_pic)


# init
rec_txt.value = RECORD
read_rec()
display_com()

HBox(children=(Text(value='./label.txt', description='RECORD:', layout=Layout(width='90%')), Button(button_style='primary', description='Read File', layout=Layout(width='80px'), style=ButtonStyle())))

Dropdown(description='[1/9456]:', layout=Layout(width='90%'), options=(('./data/samples/0.jpg', 0), ('./data/samples/1.jpg', 1), ('./data/samples/10.jpg', 2), ('./data/samples/11.jpg', 3), ('./data/samples/11.jpg', 4), ('./data/samples/12.jpg', 5), ('./data/samples/13.jpg', 6), ('./data/samples/13.jpg', 7), ('./data/samples/14.jpg', 8), ('./data/samples/15.jpg', 9), ('./data/samples/16.jpg', 10), ('./data/samples/17.jpg', 11), ('./data/samples/17.jpg', 12), ('./data/samples/18.jpg', 13), ('./data/samples/18.jpg', 14), ('./data/samples/18.jpg', 15), ('./data/samples/19.jpg', 16), ('./data/samples/19.jpg', 17), ('./data/samples/2.jpg', 18), ('./data/samples/2.jpg', 19), ('./data/samples/20.jpg', 20), ('./data/samples/20.jpg', 21), ('./data/samples/21.jpg', 22), ('./data/samples/21.jpg', 23), ('./data/samples/23.jpg', 24), ('./data/samples/24.jpg', 25), ('./data/samples/24.jpg', 26), ('./data/samples/24.jpg', 27), ('./data/samples/25.jpg', 28), ('./data/samples/26.jpg', 29), ('./data/samples/26.jpg', 30), ('./data/samples/26.jpg', 31), ('./data/samples/27.jpg', 32), ('./data/samples/27.jpg', 33), ('./data/samples/28.jpg', 34), ('./data/samples/28.jpg', 35), ('./data/samples/29.jpg', 36), ('./data/samples/29.jpg', 37), ('./data/samples/3.jpg', 38), ('./data/samples/31.jpg', 39), ('./data/samples/32.jpg', 40), ('./data/samples/33.jpg', 41), ('./data/samples/33.jpg', 42), ('./data/samples/34.jpg', 43), ('./data/samples/35.jpg', 44), ('./data/samples/35.jpg', 45), ('./data/samples/36.jpg', 46), ('./data/samples/37.jpg', 47), ('./data/samples/39.jpg', 48), ('./data/samples/39.jpg', 49), ('./data/samples/3_1.jpg', 50), ('./data/samples/3_1.jpg', 51), ('./data/samples/3_100.jpg', 52), ('./data/samples/3_1000.jpg', 53), ('./data/samples/3_1001.jpg', 54), ('./data/samples/3_1001.jpg', 55), ('./data/samples/3_1003.jpg', 56), ('./data/samples/3_1007.jpg', 57), ('./data/samples/3_1007.jpg', 58), ('./data/samples/3_1012.jpg', 59), ('./data/samples/3_1013.jpg', 60), ('./data/samples/3_1014.jpg', 61), ('./data/samples/3_1014.jpg', 62), ('./data/samples/3_1014.jpg', 63), ('./data/samples/3_1016.jpg', 64), ('./data/samples/3_1017.jpg', 65), ('./data/samples/3_1020.jpg', 66), ('./data/samples/3_1021.jpg', 67), ('./data/samples/3_1022.jpg', 68), ('./data/samples/3_1022.jpg', 69), ('./data/samples/3_1026.jpg', 70), ('./data/samples/3_1029.jpg', 71), ('./data/samples/3_1030.jpg', 72), ('./data/samples/3_1030.jpg', 73), ('./data/samples/3_1033.jpg', 74), ('./data/samples/3_1033.jpg', 75), ('./data/samples/3_1034.jpg', 76), ('./data/samples/3_1034.jpg', 77), ('./data/samples/3_1034.jpg', 78), ('./data/samples/3_1035.jpg', 79), ('./data/samples/3_1035.jpg', 80), ('./data/samples/3_1037.jpg', 81), ('./data/samples/3_1038.jpg', 82), ('./data/samples/3_1039.jpg', 83), ('./data/samples/3_1039.jpg', 84), ('./data/samples/3_1042.jpg', 85), ('./data/samples/3_1043.jpg', 86), ('./data/samples/3_1044.jpg', 87), ('./data/samples/3_1044.jpg', 88), ('./data/samples/3_1047.jpg', 89), ('./data/samples/3_1048.jpg', 90), ('./data/samples/3_1048.jpg', 91), ('./data/samples/3_105.jpg', 92), ('./data/samples/3_105.jpg', 93), ('./data/samples/3_105.jpg', 94), ('./data/samples/3_1050.jpg', 95), ('./data/samples/3_1050.jpg', 96), ('./data/samples/3_1051.jpg', 97), ('./data/samples/3_1051.jpg', 98), ('./data/samples/3_1052.jpg', 99), ('./data/samples/3_1054.jpg', 100), ('./data/samples/3_1054.jpg', 101), ('./data/samples/3_1056.jpg', 102), ('./data/samples/3_1056.jpg', 103), ('./data/samples/3_1057.jpg', 104), ('./data/samples/3_106.jpg', 105), ('./data/samples/3_106.jpg', 106), ('./data/samples/3_1073.jpg', 107), ('./data/samples/3_1074.jpg', 108), ('./data/samples/3_1076.jpg', 109), ('./data/samples/3_1077.jpg', 110), ('./data/samples/3_1081.jpg', 111), ('./data/samples/3_1081.jpg', 112), ('./data/samples/3_1083.jpg', 113), ('./data/samples/3_1084.jpg', 114), ('./data/samples/3_1085.jpg', 115), ('./data/samples/3_1088.jpg', 116), ('./data/samples/3_1089.jpg', 117), ('./data

HBox(children=(Button(description='Preview', disabled=True, style=ButtonStyle()), Button(description='Next', style=ButtonStyle()), Button(description='AutoPlay', layout=Layout(width='200px'), style=ButtonStyle()), Button(description='Save', style=ButtonStyle())))

HTML(value='<font color="#00ff0000">Saved into ./label.txt</font>')

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x01\x01\x01\x01\x02\x01\x01\x01\x02\x02\x02\x02\x02\x04\x03\x02\x02\x02\x02\x05\x04\x04\x03\x04\x06\x05\x06\x06\x06\x05\x06\x06\x06\x07\t\x08\x06\x07\t\x07\x06\x06\x08\x0b\x08\t\n\n\n\n\n\x06\x08\x0b\x0c\x0b\n\x0c\t\n\n\n\xff\xdb\x00C\x01\x02\x02\x02\x02\x02\x02\x05\x03\x03\x05\n\x07\x06\x07\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\xff\xc0\x00\x11\x08\x01h\x01P\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x18\x19\x1a%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x83\x84\x85\x86\x87\x88\x89\x8a\x92\x93\x94\x95\x96\x97\x98\x99\x9a\xa2\xa3\xa4\xa5\xa6\xa7\xa8\xa9\xaa\xb2\xb3\xb4\xb5\xb6\xb7\xb8\xb9\xba\xc2\xc3\xc4\xc5\xc6\xc7\xc8\xc9\xca\xd2\xd3\xd4\xd5\xd6\xd7\xd8\xd9\xda\xe1\xe2\xe3\xe4\xe5\xe6\xe7\xe8\xe9\xea\xf1\xf2\xf3\xf4\xf5\xf6\xf7\xf8\xf9\xfa\xff\xc4\x00\x1f\x01\x00\x03\x01\x01\x01\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x11\x00\x02\x01\x02\x04\x04\x03\x04\x07\x05\x04\x04\x00\x01\x02w\x00\x01\x02\x03\x11\x04\x05!1\x06\x12AQ\x07aq\x13"2\x81\x08\x14B\x91\xa1\xb1\xc1\t#3R\xf0\x15br\xd1\n\x16$4\xe1%\xf1\x17\x18\x19\x1a&\'()*56789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x82\x83\x84\x85\x86\x87\x88\x89\x8a\x92\x93\x94\x95\x96\x97\x98\x99\x9a\xa2\xa3\xa4\xa5\xa6\xa7\xa8\xa9\xaa\xb2\xb3\xb4\xb5\xb6\xb7\xb8\xb9\xba\xc2\xc3\xc4\xc5\xc6\xc7\xc8\xc9\xca\xd2\xd3\xd4\xd5\xd6\xd7\xd8\xd9\xda\xe2\xe3\xe4\xe5\xe6\xe7\xe8\xe9\xea\xf2\xf3\xf4\xf5\xf6\xf7\xf8\xf9\xfa\xff\xda\x00\x0c\x03\x01\x00\x02\x11\x03\x11\x00?\x00\xfb\x92\x8a*J\xe33\n*J+0#\xa8\xea\xc5IY\x81N\x8a\xb1Ev\x01\x1d\x15%I@\x15\xeaJ\xb1E\x00\x15%\x15b\xb8\xcd\x08\xe8\xa9(\xae\xc30\xa2\x8a+\x9c\xcc(\xa2\xa4\xac\xcd\x08\xe8\xa9(\xa0\x02\xa4\xab\x15%t\x00U\xca\xafV+C2\xc5X\xaa\xf5r\xb33\n\x92\xa3\xa2\xb44,W\x0f\xf1\x93\xfc+\xb4\xac\xbf\x19x7\xfe(J\x0c\xcf#\xa8\xeb>\xa3\xafp\xe7.Tu^\x8a\x00\xb9Q\xd5z(3,Uz\x8e\xa3\xa0\t*:\x8e\xa3\xa0\x0fD\xa9*:+\xe5\xcfP\xb1EGRP\x05\x8a*\xbd\x14\x00QE\x15\xd0\x04\x95%GRP\x01RQRV`IRTtV`\x14QEh\x04\x94TtVfd\x95%GV(4\n\x92\x8a(\x02\xc5\x15\x1dX\xa0\x02\xa4\xa8\xeaJ\xd0\xe7$\xab\x95N\xa4\xac\xc0\xb9E\x15%h\x01Z\x15\x9fRV\x80|\xef\xe3\x1f\xf9\x1e\xaa:\xeb\x7fi\x7f\xf9\x1f\x07\xd6\xb8j\xfa\x00$\xa2\xa3\xa2\x80$\xa2\xa3\xa8\xe88\xc9*\x9d\x15\x1d\x07@QEG@\x1e\xa9Ey\xff\x00\xfc.o\x02\xd1\xff\x00\x0b\x9b\xc0\xb5\xf2\xe7a\xe8\x15%y\xdf\xfc4\xbf\x80\xfd\x07\xe7Q\xff\x00\xc3K\xf8\x0f\xd0~t\x01\xe9\x94W\x9f\xff\x00\xc2\xe4\xf6\xfd*\xc7\xfc.Oo\xd2\x80;\xca+\x83\xff\x00\x85\xc9\xed\xfaT\x7f\xf0\xb9=\xbfJ\xd0\x0fD\xa9+\xcd\xff\x00\xe1r{~\x95\'\xfc.Oo\xd2\xb3\x03\xd2*:\xf3\xbf\xf8\\\x9e\xdf\xa5G\xff\x00\x0b\x93\xdb\xf4\xa0\x0fL\xa9+\xcb\xff\x00\xe1r{~\x95\'\xfc.Oo\xd2\x80=2\x8a\xf3\x7f\xf8\\\x9e\xdf\xa5S\xff\x00\x85\xc9\xed\xfaP\x07\xaaQ^_\xff\x00\x0b\x93\xdb\xf4\xa3\xfe\x17\'\xb7\xe9@\x1e\xa9RW\x97\xff\x00\xc2\xe4\xf6\xfd*\xe7\xfc.Oo\xd2\x80=B\xa4\xaf\x9d\xfce\xff\x00\x05,\xf8\x13\xf0o\xfej%|\xff\x00\xe3/\xf88\x9b\xe0M\x00~\x84T\x95\xf9\x97\xff\x00\x11\x12\xfbW\xa6x7\xfe\x0b\xe9\xf0\'\xc6T\x01\xf7%I_?\xff\x00\xc3\xcb>\x04\xf8\xc4\xff\x00\xc9D\xae\x93\xfe\x1a_\xc0~\x83\xf3\xa0\xcc\xf6\n\x92\xbc3\xfe\x1b\x1f\xc0~\x87\xf3\xaa\xff\x00\xf0\xd8\xfe\x03\xf4?\x9d\x06g\xd1u%x\xdf\xfc4\xbf\x80\xfd\x07\xe7Y\xff\x00\xf0\xd2\xfe\x03\xf4\x1f\x9d\x00{\xa5I^\x17\xff\x00\r/\xe0?A\xf9\xd4\x9f\xf0\xd2\xfe\x03\xf4\x1f\x9dv\x1a\x1d\xc7\xed-\xe0\xdf\xf8\xa1+\xc5\xeb\xd2<\x19\xf1\x93\xc0\x9e2\xf0\'\x88\xab\xcb\xeb\xd0\x02\xc5GEG]\x86aV*\xbdG@\x12Tu^\x8a\x00\xb1U\xe8\xaa\xf5\xce\x05\x9f\xf8`\x7f\x01\xff\x00\xd4\xc7O\xff\x00\x86\x03\xf0/\xadr\x7f\xf0\xff\x00O\

HBox(children=(Text(value='', description='New label:'), Label(value='Preview Label: 1')))